In [ ]:
#@title ##Upgrading the Generative Ai liberary to its latest model.
!pip install google-generativeai --upgrade

# Configuring the Gemini API key

In [ ]:
#@title Importing the Google Gemini AI Software Development Kit __(google.generativeai)__ and assigning it an alias __genai__

import google.generativeai as genai

In [ ]:
#@title Importing the <b>userdate</b> from __google.colob__, which is used for handing user specifi data, Such as storing and retrieving the API keys securely in colab.
from google.colab import userdata

In [ ]:
#@title Importing __types__ from __google.genereativeai__ which will handles __data types__ for the API.
from google.generativeai import types

In [ ]:
#@title Defining the API key name __GOOGLE_API_KEY__ which is being stored in the variable.
#@markdown ######API keys are stored securely. This string __(GOOGLE_API_KEY)__ acts as a reference name to retrieve the actual API key from the source.
GEMINI_API_NAME = "GOOGLE_API_KEY" #@param{type: "string"}
#@markdown ###### Paramter __#@param{type: "string"}__ is another Colob-formatted comment used to create an interatve input field in the notebook. Which is as shown above.
#@markdown ###### __type: "string"__ is used so that string value can be used for understanding

In [ ]:

# This is the name used to store in colab environment.

try:
  GOOGLE_API_KEY = userdata.get(GEMINI_API_NAME)# Retrieves the API key form the stored secret.
  genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
except userdata.SecretNotFoundError as e:
  print(f"Secert not fount\n\nThis expects you to create a secret named {GEMINI_API_NAME} in Colab\n\nVisit https://makesuite.google.com/app/apikey to create and API key\n\nStore that is the secrets section on the left side of the notebook (key icon)\n\nName the secret{GEMINI_API_NAME}")
  raise e

In [ ]:

except userdata.NotebookAccessError as e:
  print(f"You need to grant this notebook access to the {GEMINI_API_NAME} secret in order for the notebook to acces Gemini on your behalf.")


In [ ]:
except Exception as e:
  #unknown error
  print(f"There was an uknow error. Ensure you have a secret {GEMINI_API_NAME} stored in Colab and it's a vaild key from https://makersuite.google.com/app/apikey")
  raise e


In [ ]:

model = genai.GenerativeModel("gemini-pro-vision")